In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pythainlp python-crfsuite

     |████████████████████████████████| 11.5 MB 7.8 MB/s 
     |████████████████████████████████| 743 kB 71.3 MB/s 


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import InputLayer , Embedding , LSTM , TimeDistributed , Dense , Activation, Bidirectional,Concatenate, Input, Dropout
import glob
import os
import sys
from pythainlp.tokenize import word_tokenize, sent_tokenize
from pythainlp.tag import pos_tag,pos_tag_sents
from pythainlp.tag.thainer import ThaiNameTagger
from pythainlp import tag
from tqdm import tqdm
import re
import copy

In [2]:
st = open('/content/drive/MyDrive/SuperAI_NLP/ss_test.txt','r').read().split('\n')
st = '_'.join(st)
st = st.split('_')
max([len(i) for i in st])

36

In [5]:
!tar -xf /content/drive/MyDrive/SuperAI_NLP/AIFORTHAI-LST20Corpus.tar.gz
!wget https://raw.githubusercontent.com/KoichiYasuoka/spaCy-Thai/master/th_pud-ud-orchid.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Thai-PUD/master/th_pud-ud-test.conllu

--2022-02-18 12:10:00--  https://raw.githubusercontent.com/KoichiYasuoka/spaCy-Thai/master/th_pud-ud-orchid.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647521 (1.6M) [text/plain]
Saving to: ‘th_pud-ud-orchid.conllu’

th_pud-ud-orchid.co 100%[===================>]   1.57M  --.-KB/s    in 0.03s   

2022-02-18 12:10:00 (47.6 MB/s) - ‘th_pud-ud-orchid.conllu’ saved [1647521/1647521]

--2022-02-18 12:10:00--  https://raw.githubusercontent.com/UniversalDependencies/UD_Thai-PUD/master/th_pud-ud-test.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaitin

### **Preprocess orchid - PUD dataset**

In [3]:
orchid = open('/content/th_pud-ud-orchid.conllu','r').read()
pud = open('/content/th_pud-ud-test.conllu','r').read()
orchid = orchid + pud
orchid = orchid.split('\n')
orchid = list(filter(lambda x: len(x) != 0, orchid))

orchid = list(filter(lambda x: x[0].isdigit(), orchid))
orchid = [x.split('\t') for x in orchid]

orchid_sentences = []
prev = 0
temp = []
for i in orchid:
  num = int(i[0])
  if num < prev:
    orchid_sentences.append(temp)
    temp = []
    temp.append(i[1])
  else: temp.append(i[1])
  prev = num

orchid_sentences.append(temp)
orchid_sentences

print(f'word : {len([j for i in orchid_sentences for j in i])}')

def get_sentence_pair_and_label_orchid(sentences):
  sentence_pairs = []
  label = [1]*len(sentences)
  for i in range(len(sentences)-1):
    sentence_pairs.append([sentences[i], sentences[i+1]])

  return [sentence_pairs], [label]


word : 44656


In [4]:
train_path = '/content/LST20_Corpus/train'
test_path = '/content/LST20_Corpus/test'

def get_doc_old(path):
  docs = []
  for i in tqdm(list(os.scandir(path))):
    if i.name[0] != '.':
      doc = open(i.path,'r').read()
      doc = doc.split('\n\n')
      doc = [[k.split('\t')[0] for k in d.split('\n')] for d in doc]
      docs.append(doc)
  return docs

def get_doc(path):
  docs = []
  for i in tqdm(list(os.scandir(path))):
    if i.name[0] != '.':
      doc = open(i.path,'r').read()
      doc = doc.split('\n')
      doc = [d.split('\t')[0] if d != '' else '<SEP>' for d in doc]
      docs.append(doc[:-2])
  return docs

docs = get_doc(train_path)
docs_test = get_doc(test_path)


def get_sentence_pair_and_label(docs):
  X,Y = [],[]
  for doc in docs:
    sentence_pairs = []
    labels = []
    sen_pair = []
    label = []
    idx_to_cut = [0]*len(doc)
    for i in range(len(doc)):
      if doc[i] == '<SEP>':
        idx_to_cut[i] = 1
    
    word_seg = []
    ws = []
    for word in doc:
      if word == '<SEP>' or word == '_':
        word_seg.append(ws)
        ws = []
      else:
        ws.append(word)
    word_seg.append(ws)

    word_seg = [w + ['_'] for w in word_seg]

    sum_len = 0
    size = []
    for sub in range(len(word_seg)-1):
      len_sub = len(word_seg[sub]) + sum_len - 1
      is_cut = idx_to_cut[len_sub]
      labels.append(is_cut)
      sentence_pairs.append([word_seg[sub], word_seg[sub+1]])
      sum_len += len(word_seg[sub])

    X.append(sentence_pairs)
    Y.append(labels)

  return X,Y

ner = ThaiNameTagger()
def get_ner(seg,ner):
  gt = seg
  txt = ner.get_ner(''.join(gt))
  result = []
  for i in txt:
    tag = i[2]
    for char in i[0]:
      result.append([char,tag])
  result
  gt_idx = [len(i) for i in gt][:-1]
  count = 0
  segs = []
  for i in gt_idx:
    to_cut = i + count
    seg = result[count:i+count]
    segs.append(seg)
    count += i
  seg = result[count:i+count]
  segs.append(seg)

  result = []
  for i in segs:
    tag = i[0][1]
    result.append(tag)

  return result

docs_test = get_doc(test_path)

list_pos = ['AJ', 'AV', 'AX', 'CC', 'CL', 'FX', 'IJ', 'NG', 'NN', 'NU', 'PA', 'PR', 'PS', 'PU', 'VV', 'XX']
list_ner = ['DATE','TIME','EMAIL' ,'LEN' ,'LOCATION' ,'ORGANIZATION' ,'PERSON' ,'PHONE' ,'URL', 'ZIP' ,'MONEY' ,'LAW', 'PERCENT','O']
dic_pos = {j:i for i,j in enumerate(list_pos)}
dic_ner = {j:i for i,j in enumerate(list_ner)}
dic_pos['<PAD_TOKEN>'] = len(dic_pos)
dic_ner['<PAD_TOKEN>'] = len(dic_ner)
inv_dic_pos = {i:j for j,i in dic_pos.items()}
inv_dic_ner = {i:j for j,i in dic_ner.items()}


orchid_dict = set([j for i in orchid_sentences for j in i])
dic = {j:i for i,j in enumerate(list(set([x for i in get_doc_old(train_path) for d in i for x in d]).union(orchid_dict)))}
dic_class = {}
dic['<PAD_TOKEN>'] = len(dic)
dic['<UNKNOWN>'] = len(dic)
inv_dic = {i:j for j,i in dic.items()}
dic_class['I_SENT'] = len(dic_class)
dic_class['B_SENT'] = len(dic_class)
dic_class['E_SENT'] = len(dic_class)
dic_class['O_SENT'] = len(dic_class)
dic_class['P_SENT'] = len(dic_class)
inv_dic_class = {i:j for j,i in dic_class.items()}

sentence_pairs, labels = get_sentence_pair_and_label(docs)
sentence_pairs_orchid, labels_orchid = get_sentence_pair_and_label_orchid(orchid_sentences)
sentence_pairs_test, labels_test = get_sentence_pair_and_label(docs_test)

max_len = 50
def preprocess_sentence(x):
  output = []
  x_ner = get_ner(x,ner)
  x_ner = [x if x == 'O' else x.split('-')[1] for x in x_ner]
  x = [[i, dic_pos[pos_tag([i], corpus='lst20')[0][1]], dic_ner[xner]] for i, xner in zip(x,x_ner)]
  x = [[dic[i[0]] if i[0] in dic.keys() else dic['<UNKNOWN>'], i[1], i[2]] for i in x]
  if len(x) > max_len: return -1
  x =  x + [[dic['<PAD_TOKEN>'], dic_pos['<PAD_TOKEN>'], dic_ner['<PAD_TOKEN>']]]*(max_len - len(x))
  return x

def gen_data(sentence_pairs, labels):
  X_train = []
  Y_train = []
  for sentence_pair,label in tqdm(zip(sentence_pairs, labels), total=len(sentence_pairs)):
    temp = []
    temp_l = []
    s2_copy = None
    for (s1,s2) ,lab in zip(sentence_pair, label):
      s1 = s2_copy if s2_copy is not None else preprocess_sentence(s1)
      s2 = preprocess_sentence(s2)
      s2_copy = copy.deepcopy(s2)
      if s1 == -1 or s2 == -1: continue
      temp.append([s1,s2])
      temp_l.append(lab)
    X_train.append(temp)
    Y_train.append(temp_l)
  return X_train, Y_train

X_train, Y_train  = gen_data(sentence_pairs, labels)
X_train_orchid, Y_train_orchid  = gen_data(sentence_pairs_orchid, labels_orchid)
X_test, Y_test  = gen_data(sentence_pairs_test, labels_test)


100%|██████████| 483/483 [00:18<00:00, 25.96it/s]


In [5]:
# for i,j,k in X_train[0][20][0]:
#   print(inv_dic[i],j,inv_dic_ner[k])

In [26]:
# X_train_2 = []
merged_X_train = []
merged_Y_train = []
merged_X_test = []
merged_Y_test = []
for i,j in zip(X_train,Y_train):
  merged_X_train.extend(i)
  merged_Y_train.extend(j)

for i,j in zip(X_train_orchid,Y_train_orchid):
  merged_X_train.extend(i)
  merged_Y_train.extend(j)

for i,j in zip(X_test,Y_test):
  merged_X_test.extend(i)
  merged_Y_test.extend(j)
    # print(np.array(j))
  # print(np.array(i).shape)

In [7]:
len([j for i in Y_train for j in i])

461192

In [27]:
merged_X_train = np.array(merged_X_train)
merged_Y_train = np.array(merged_Y_train)
merged_X_test = np.array(merged_X_test)
merged_Y_test = np.array(merged_Y_test)

### **LST-20 Dataset**

In [28]:
print(merged_X_train.shape, merged_Y_train.shape)
print(merged_X_test.shape, merged_Y_test.shape)

(463167, 2, 50, 3) (463167,)
(37955, 2, 50, 3) (37955,)


**LST-20 Dataset + Orchid**

In [10]:
print(merged_X_train.shape, merged_Y_train.shape)
print(merged_X_test.shape, merged_Y_test.shape)

(463167, 2, 50, 3) (463167,)
(37955, 2, 50, 3) (37955,)


**Augment negative class**

In [29]:
import random
perm = [[p1,p2] for p1,p2 in zip(np.random.permutation(len(merged_X_train)),np.random.permutation(len(merged_X_train)))]
augmented_X_train = []
augmented_Y_train = []
for p1,p2 in tqdm(perm):
  if p1 == p2: continue
  dat1 = merged_X_train[p1]
  dat2 = merged_X_train[p2]
  sen1 = dat1[0,:,:]
  sen2 = dat2[1,:,:]
  sen3 = dat1[1,:,:]
  sen4 = dat2[0,:,:]
  aug1 = np.array([sen1,sen2])
  aug2 = np.array([sen3,sen4])
  augmented_X_train.extend([aug1, aug2])
  augmented_Y_train.extend([0,0])

100%|██████████| 463167/463167 [00:03<00:00, 139406.11it/s]


In [12]:
augmented_X_train = np.array(augmented_X_train)
augmented_Y_train = np.array(augmented_Y_train)
merged_X_train = np.concatenate((merged_X_train, augmented_X_train), axis = 0)
merged_Y_train = np.concatenate((merged_Y_train, augmented_Y_train), axis = 0)


In [13]:
input_word = Input(shape=(50,))
embed_word = Embedding(len(dic),64)(input_word)
input_pos = Input(shape=(50,))
embed_pos = Embedding(len(dic_pos),10)(input_pos)
input_ner = Input(shape=(50,))
embed_ner = Embedding(len(dic_ner),20)(input_ner)
input_word2 = Input(shape=(50,))
embed_word2 = Embedding(len(dic),64)(input_word2)
input_pos2 = Input(shape=(50,))
embed_pos2 = Embedding(len(dic_pos),10)(input_pos2)
input_ner2 = Input(shape=(50,))
embed_ner2 = Embedding(len(dic_ner),20)(input_ner2)

concat = Concatenate()([embed_word, 
                        embed_pos, 
                        embed_ner, 
                        embed_word2, 
                        embed_pos2, 
                        embed_ner2])

bidirec = Bidirectional(LSTM(100,return_sequences=False))(concat)
time_dis = Dense(30)(bidirec)
drop = Dropout(0.3)(time_dis)
dense = Dense(13)(drop)
dense = Dense(1)(dense)
act = Activation('sigmoid')(dense)
model = Model(inputs=[input_word, input_pos, input_ner, input_word2, input_pos2, input_ner2], outputs=[act])
model.compile(optimizer="Adam",loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'], run_eagerly=True)

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 50)]         0           []                               
                                                                                              

In [15]:
class_weight = {0: 1.1,
                1: 15.0,
                }
class_weight

{0: 1.1, 1: 15.0}

In [16]:
def gen_sample_weight(Y, class_weight):
  Y = copy.deepcopy(np.array(Y).astype(np.float32))
  for k,v in class_weight.items():
    Y[Y==k] = v
  return Y

sample_weight = gen_sample_weight(merged_Y_train, class_weight)

In [17]:
sample_weight

array([ 1.1, 15. ,  1.1, ..., 15. , 15. , 15. ], dtype=float32)

In [41]:
model.fit(
    [merged_X_train[:,0,:,0],
     merged_X_train[:,0,:,1],
     merged_X_train[:,0,:,2],
     merged_X_train[:,1,:,0],
     merged_X_train[:,1,:,1],
     merged_X_train[:,1,:,2],
     ], 
    merged_Y_train, 
    batch_size = 500, 
    validation_data = ( 
      [merged_X_test[:,0,:,0],
     merged_X_test[:,0,:,1],
     merged_X_test[:,0,:,2],
     merged_X_test[:,1,:,0],
     merged_X_test[:,1,:,1],
     merged_X_test[:,1,:,2],
     ], merged_Y_test),
    # sample_weight = gen_sample_weight(merged_Y_train, class_weight), 
    epochs=1)

927/927 [==============================] - 25s 27ms/step - loss: 0.1041 - accuracy: 0.9597 - val_loss: 0.2890 - val_accuracy: 0.9055


In [ ]:
merged_X_train[:,0,:,0]

array([[33838,  7361, 22217, ..., 48946, 48946, 48946],
       [38072, 19040, 48946, ..., 48946, 48946, 48946],
       [20063, 19040, 48946, ..., 48946, 48946, 48946],
       ...,
       [ 4357, 23543, 44581, ..., 48946, 48946, 48946],
       [31821, 19040, 48946, ..., 48946, 48946, 48946],
       [12616, 19040, 48946, ..., 48946, 48946, 48946]])

In [58]:
y_pred = model.predict([merged_X_test[:,0,:,0],
     merged_X_test[:,0,:,1],
     merged_X_test[:,0,:,2],
     merged_X_test[:,1,:,0],
     merged_X_test[:,1,:,1],
     merged_X_test[:,1,:,2],
     ])

In [37]:
model.save('/content/drive/MyDrive/SuperAI_NLP/binary_cut_5epoch_92f1.h5')

In [ ]:
correct = 0
total = np.sum(np.array(Y_test) == dic_class['O_SENT'])
corr = 0
for yp,yt in zip(y_pred, Y_test):
  pred = [inv_dic_class[d] for d in np.argmax(yp, axis=1).tolist()]
  gt = [inv_dic_class[d] for d in yt]

  for p,g in zip(pred, gt):
    if g == 'O_SENT':
      if p == g: corr += 1
      # print(p,g)
  # break

print('ac', corr/total)

ac 0.7570367474589523


In [46]:
import pandas as pd

my_file =  open('/content/drive/MyDrive/SuperAI_NLP/ss_test.txt', 'r')


r = my_file.read()


data = r.split("\n")
# print(data)
my_file.close()
text = ''
for  i in range(len(data)):
    
    if data[i] =='':
        text = text+' '
        data[i] = '_'
    else:
        text = text+data[i]
# print(text)
# print(data)
df = pd.DataFrame({'word':data})
data = np.array(data)
id = np.where(data == '_')[0]


In [47]:
data = '<separator>'.join(data).split('<separator>')

temp_data = []
t = []
for i in data:
  if i != '_': t.append(i)
  else: temp_data.append(t); t = []
if len(t) != 0: temp_data.append(t); t=[] 

data = temp_data
sent_pair = []
for i in range(len(data)-1):
  if data[i].__len__() == 0: break
  sent1 = data[i]
  sent2 = data[i+1]
  if len(sent2) == 0: continue
  # if sent1[0] == '' and len(sent1) > 1: sent1 = sent1[1:]
  # if sent2[0] == '' and len(sent2) > 1: sent2 = sent2[1:]
  sent1 = sent1[:] + ['_']
  sent2 = sent2[:] + ['_']
  pos1 = [dic_pos[pos_tag([k], corpus='lst20')[0][1]] for k in sent1]
  pos2 = [dic_pos[pos_tag([k], corpus='lst20')[0][1]] for k in sent2]
  ner1 = get_ner(sent1,ner)
  ner2 = get_ner(sent2,ner)
  ner1 = [x if x == 'O' else x.split('-')[1] for x in ner1]
  ner2 = [x if x == 'O' else x.split('-')[1] for x in ner2]
  ner1 = [dic_ner[x] for x in ner1]
  ner2 = [dic_ner[x] for x in ner2]
  sent1 = [dic[d] if d in dic.keys() else dic['<UNKNOWN>'] for d in sent1]
  sent2 = [dic[d] if d in dic.keys() else dic['<UNKNOWN>'] for d in sent2]
  pos1 = pos1 + [dic_pos['<PAD_TOKEN>']]*(50 - len(pos1))
  pos2 = pos2 + [dic_pos['<PAD_TOKEN>']]*(50 - len(pos2))
  ner1 = ner1 + [dic_ner['<PAD_TOKEN>']]*(50 - len(ner1))
  ner2 = ner2 + [dic_ner['<PAD_TOKEN>']]*(50 - len(ner2))
  sent1 = sent1 + [dic['<PAD_TOKEN>']]*(50 - len(sent1))
  sent2 = sent2 + [dic['<PAD_TOKEN>']]*(50 - len(sent2))
  merg1 = [[d,p,n] for d,p,n in zip(sent1,pos1,ner1)]
  merg2 = [[d,p,n] for d,p,n in zip(sent2,pos2,ner2)]
  sent_pair.append([merg1,merg2])
# data = [[d, pos_tag([i], corpus='lst20')[0][1]] for d in data]
  # prdint(pred)
  # index = np.sort(id[(id > inc) & (id < 1024 + inc)])[-1]
  # inc = i
  # print(index)
sent_pair = np.array(sent_pair)

In [ ]:
data[i]

['2/2004']

In [ ]:
data[-3]

['หนังสือพิมพ์', 'ผู้', 'จัด', 'การ']

In [ ]:
sent_pair[11]

array([[[11414,     8],
        [30003,     8],
        [36247,     8],
        [31124,     8],
        [21452,     9],
        [ 4532,    12],
        [  911,     8],
        [27797,     8],
        [38130,     8],
        [12227,     3],
        [32948,     8],
        [ 3709,    14],
        [ 3711,    12],
        [ 9210,     8],
        [27797,     8],
        [40095,    13],
        [22949,    13],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946,    16],
        [48946, 

In [49]:
out = np.round(model.predict(
   [sent_pair[:,0,:,0],
    sent_pair[:,0,:,1],
     sent_pair[:,0,:,2],
     sent_pair[:,1,:,0],
     sent_pair[:,1,:,1],
     sent_pair[:,1,:,2],]
))

In [50]:
group_output = []
for d,o in zip(data, out):
  group_output.append([d,o])

In [51]:
out_result = []
lab = []

flag = True
for i in group_output:
  # i[0] = ''i[0].strip(' ')
  if flag : lab.append('B_SENT'); flag=False
  else: lab.append('I_SENT')
  # i[0] = i[0].split(' ')
  out_result.append(i[0][0])
  # if len(i[0]) > 1:
  for j in i[0][1:]:
    lab.append('I_SENT')
    out_result.append(j)
  if i[1][0] == 1.:
    out_result.append('_')
    lab[-1] = 'E_SENT'
    lab.append('O')
    flag = True
  else: 
    out_result.append('_')
    # lab[-1] = 'I_SENT'
    lab.append('I_SENT')


In [52]:
t = []
for i,j,l in zip(out_result, data,lab):
  t.append([i,j,l])


In [53]:
lab.append('I_SENT')

In [56]:
sub['Predicted'] = lab

In [ ]:
data[::-1][:100]

array(['_', '_', '2/2004', '_', 'การ', 'จัด', 'ผู้', 'หนังสือพิมพ์', '_',
       'ใหม่', 'อุบัติ', 'โรค', 'ควบคุม', 'การ', 'ถึง', 'ไป', 'รวม', '_',
       'เร็ว', 'ได้', 'โรค', 'ของ', 'ระบาด', 'การ', 'ควบคุม', 'และ',
       'ดูแล', 'ช่วย', 'จะ', '_', 'ได้', 'ศูนย์', '_', '12', '_', 'ทั้ง',
       'ภูมิภาค', 'ใน', 'แล็บ', 'ศูนย์', 'ของ', 'เข้มแข็ง', 'ความ',
       'พัฒนา', 'สามารถ', 'ถ้า', 'ซึ่ง', '_', 'ยาว', 'คิว', 'วิเคราะห์',
       'ตรวจ', 'การ', 'ที่', 'ขณะ', '_', 'กรุงเทพฯ', 'ใน', 'เฉพาะ',
       'เข้มแข็ง', 'ศูนย์', 'มี', 'เหมือน', 'นี้', 'ขณะ', 'เพราะ', '_',
       'เขต', 'ศูนย์', 'ใน', '_', 'จังหวัด', 'ต่าง', 'เขต', 'ใน', 'แล็บ',
       'ศักยภาพ', 'พัฒนา', 'การ', 'เรื่อง', 'พิจารณา', 'กำลัง', 'ยาว',
       'ระยะ', 'แต่', '_', 'แล้ว', 'หนึ่ง', 'ส่วน', 'ไว้', 'ขอ', 'นี้',
       'ขณะ', 'นั้น', 'ประมาณ', 'งบ', 'เรื่อง', '_', 'ว่า', 'กล่าว'],
      dtype='<U36')

In [57]:
sub.to_csv('submitzz.csv',index=False)

In [ ]:
data[:100]

array(['กล่าว', 'ชม', 'ตร.', 'เร่ง', 'สำนวน', 'คดี', 'เร็ว', 'ดี', 'ลั่น',
       'จะ', 'ไม่', 'หนี', 'เหมือน', 'นัก', 'การ', 'เมือง', 'บาง', 'คน',
       '_', 'ตาม', 'ที่', 'นาย', 'ธนชาติ', '_', 'แสงประดับธรรมโชติ', '_',
       'เข้า', 'แจ้ง', 'ความ', 'กับ', '_', 'ตำรวจ', '_', 'สน.',
       'นางเลิ้ง', 'เมื่อ', '_', '29', '_', 'มิ.ย.', '_', '2551', '_',
       'โดย', 'กล่าวหา', 'นาย', 'สมเกียรติ', '_', 'พงษ์ไพบูลย์', '_',
       'ส.ส.', 'สัดส่วน', 'พรรค', 'ประชาธิปัตย์', 'หนึ่ง', 'ใน', 'แกนนำ',
       'พันธมิตร', 'ประชาชน', 'เพื่อ', 'ประชาธิปไตย', 'ปราศรัย', 'บน',
       'เวที', 'พันธมิตร', 'ฯ', '_', 'มี', 'การ', 'กล่าว', 'พาดพิง',
       'ดูหมิ่น', 'สถาบัน', '_', 'มี', 'ความ', 'ผิด', 'ตาม', 'ประมวล',
       'กฎหมาย', 'อาญา', '_', 'มาตรา', '_', '112', '_', 'โดย', 'นำ',
       'เทป', 'การ', 'กล่าว', 'ปราศรัย', '_', 'ภาพ', 'ถ่าย', '_', 'ภาพ',
       'วีดีโอ', '_', 'มอบ'], dtype='<U36')

In [ ]:
for i in data:
  print(len(i.split(' ')))

In [ ]:
print(res.__len__())
print(data.__len__())


74405
74405


In [ ]:
map_class = {
    'B_SENT':'B_SENT',
    'I_SENT':'I_SENT',
    'E_SENT':'E_SENT',
    'O_SENT':'O',
    'P_SENT':'I_SENT',
}

res = [map_class[i] for i in res]

In [ ]:
np.unique(res, return_counts=True)

(array(['B_SENT', 'E_SENT', 'I_SENT', 'O'], dtype='<U6'),
 array([ 1196,  1240, 70745,  1224]))

In [55]:
sub = pd.read_csv('/content/drive/MyDrive/SuperAI_NLP/ss_sample_submission.csv')
# sub['Predicted'] = res[:-2]

In [ ]:
sub.to_csv('submit3.csv',index=False)

In [ ]:
sub

,Id,Predicted
0,1,I_SENT
1,2,I_SENT
2,3,I_SENT
3,4,I_SENT
4,5,I_SENT
...,...,...
74398,74399,I_SENT
74399,74400,I_SENT
74400,74401,I_SENT
74401,74402,I_SENT


In [ ]:
rez = y_pred.argmax(2).flatten()

AxisError: ignored

In [ ]:
import copy
temp_rez = []
t = []
count = 0
for i in rez:
  if i == 'O':
    count += 1
    if len(t) == 0:print('t')
    temp_rez.append(copy.deepcopy(t))
    t = []
  else:
    count += 1
    t.append(i)
temp_rez.append(t)
# rez = ''.join(rez).split('O')
rez = [['I_SENT']*len(i) for i in temp_rez]
for i in range(len(rez)):
  if len(rez[i]) == 0: 
    rez[i] = ['I_SENT']
    continue
  elif len(rez[i]) == 1: rez[i] = ['B_SENT']
  elif len(rez[i]) == 2: rez[i] = ['B_SENT','E_SENT']
  else:
    rez[i][0] = 'B_SENT'
    rez[i][-1] = 'E_SENT'
  rez[i].append('O')

temp = []
for i in rez:
  for j in i:
    temp.append(j)
  # temp.append('O')
# rez = [i for j in rez for i in j]
temp = [dic_class[i] for i in temp]

KeyError: ignored

In [ ]:
from sklearn.metrics import f1_score

ytest = np.array(Y_test).flatten()
ypred = temp[:-1]

f1_score(ytest, ypred, average='micro')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


NameError: ignored

In [ ]:
from sklearn.metrics import f1_score

ytest = np.array(Y_test).flatten()
ypred = y_pred.argmax(2).flatten()

f1_score(ytest, ypred, average='micro')

0.9683412977867203

In [ ]:
from sklearn.metrics import f1_score

ytest = np.array(Y_test).flatten()
ypred = y_pred.argmax(2).flatten()

f1_score(ytest, ypred, average='micro')

0.9763286751760564

In [ ]:
from sklearn.metrics import f1_score

ytest = np.array(merged_Y_test).flatten()
ypred = np.round(y_pred[:,0]).astype(np.uint8)

f1_score(ytest, ypred, average='micro')

0.8566460281912791

In [59]:
from sklearn.metrics import f1_score

ytest = np.array(merged_Y_test).flatten()
ypred = np.round(y_pred[:,0]).astype(np.uint8)

f1_score(ytest, ypred, average='micro')

0.9200105387959425

In [36]:
from sklearn.metrics import f1_score

ytest = np.array(merged_Y_test).flatten()
ypred = np.round(y_pred[:,0]).astype(np.uint8)

f1_score(ytest, ypred, average='micro')

0.9200105387959425

In [ ]:
y_pred

array([[4.6172515e-02],
       [2.1210741e-05],
       [6.9497329e-01],
       ...,
       [2.3458363e-02],
       [2.7394648e-08],
       [5.7124515e-13]], dtype=float32)

In [42]:
model = tf.keras.models.load_model('/content/drive/MyDrive/SuperAI_NLP/binary_cut_5epoch_92f1.h5')

In [ ]:
pos_tag(docs[0][0],corpus='lst20')

[('เปิด', 'VV'),
 ('ขนส่ง', 'VV'),
 ('สินค้า', 'NN'),
 ('เชื่อม', 'VV'),
 (' ', 'PU'),
 ('3', 'NU'),
 (' ', 'PU'),
 ('ประเทศ', 'CL')]

IndexError: ignored

In [ ]:
docs[0]